In [ ]:
import os
import json
from groq import Groq
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

In [ ]:
client = Groq(api_key=os.environ["GROQ_API_KEY"])

## System prompt

In [20]:
SYSTEM_PROMPT = """\
You are part of a system that teaches programming.
Choose the function that fits best the next user message.
"""

## Functions

### Tópico

In [ ]:
TOPIC_FUNCTION_NAME = "create_topic_explanation"
TOPIC_FUNCTION_DESC = """\
Creates an explanation for a programming topic for the user.
Takes into account previous explanations in order to write clearer explanations, if the topic has already been explained before.\
"""

topic_function = {
    "type": "function",
    "function": {
        "name": TOPIC_FUNCTION_NAME,
        "description": TOPIC_FUNCTION_DESC,
        "parameters": {
            "type": "object",
            "properties": {
                "observations": {
                    "type": "string",
                    "description": "Some observations about the previous messages that should be considered when writing the explanation.",
                }
            },
            "required": ["observations"],
        },
    },
}


# TODO
def create_topic_explanation(*args, **kwargs):
    return kwargs

### Questão

In [ ]:
QUESTION_FUNCTION_NAME = "create_question"
QUESTION_FUNCTION_DESC = """\
Creates a question for the user about a topic.\
"""

QUESTION_function = {
    "type": "function",
    "function": {
        "name": QUESTION_FUNCTION_NAME,
        "description": QUESTION_FUNCTION_DESC,
        "parameters": {
            "type": "object",
            "properties": {
                "observations": {
                    "type": "string",
                    "description": "Some observations about the previous messages that should be considered when writing the explanation.",
                }
            },
            "required": ["observations"],
        },
    },
}

### Avalia questão

In [ ]:
# TODO

### Atualiza estado

In [ ]:
# TODO

### Gerar resposta

In [57]:
do_nothing_function = {
    "type": "function",
    "function": {
        "name": "response_without_tools",
        "description": "If you feel like you don't need to call a function",
        "parameters": {},
        "required": [],
    },
}

## Roteador

In [ ]:
def execute_router(client, tools, tool_names, weights):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": SYSTEM_PROMPT,
            },
            {
                "role": "user",
                "content": "Explain if else to me",
            },
        ],
        model="llama3-groq-70b-8192-tool-use-preview",
        tools=tools,
        tool_choice="required",
    )

    # TODO: Verificar argumentos
    (tool_call, _), *_ = sorted(
        [
            (call, weights[call.function.name])
            for call in chat_completion.choices[0].message.tool_calls
            if call.function.name in tool_names
        ],
        key=lambda x: x[1],
    )

    args = json.loads(tool_call.function.arguments)

    if tool_call.function.name == "do_nothing":
        tool_response = "nothing"
    elif tool_call.function.name == "create_topic_explanation":
        tool_response = create_topic_explanation(**args)

    return tool_response

In [ ]:
weights = {
    "response_without_tools": float("-inf"),
    "create_topic_explanation": 1.0,
}

tools = [topic_function, do_nothing_function]
tool_names = [tool["function"]["name"] for tool in tools]

execute_router(client=client, tools=tools, tool_names=tool_names, weights=weights)